### Lets do some feature Engineering that includes create dummy variabble on categorical data and scaling on numerical data

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [117]:
df = pd.read_csv("LendingClub_cleaned.csv")

In [118]:
df.shape

(1852324, 88)

In [119]:
#df.info()

### Select loan_status with values "Fully Paid", "Charged Off" and "Default"

In [120]:
df1 = df[(df["loan_status"] == "Fully Paid") | (df["loan_status"] == "Charged Off") | (df["loan_status"] == "Default")]

In [121]:
df1["loan_status"].value_counts()

Fully Paid     880570
Charged Off    210041
Default            30
Name: loan_status, dtype: int64

In [122]:
#df1["loan_status"]

### Convert to binary values:

In [123]:
df1["loan_status"] = df1['loan_status'].replace({'Fully Paid':0,'Default':1,'Charged Off':1})

<ipython-input-123-0a669ac3ba9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["loan_status"] = df1['loan_status'].replace({'Fully Paid':0,'Default':1,'Charged Off':1})


In [124]:
df1["loan_status"]

0          0
1          0
3          0
4          0
5          0
          ..
1852315    0
1852318    0
1852319    1
1852320    0
1852323    1
Name: loan_status, Length: 1090641, dtype: int64

In [125]:
df1.loan_status.unique()

array([0, 1], dtype=int64)

In [126]:
type(df1.term)

pandas.core.series.Series

In [127]:
df1.term = pd.to_numeric(df1.term.str.strip('months'))

C:\Users\abishek\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [128]:
df1.emp_length

0          10+ years
1          10+ years
3            3 years
4            4 years
5          10+ years
             ...    
1852315    10+ years
1852318      9 years
1852319      3 years
1852320    10+ years
1852323      6 years
Name: emp_length, Length: 1090641, dtype: object

In [129]:
df1.emp_length = df1.emp_length.str.replace('+', '')
df1.emp_length = df1.emp_length.str.replace('<', '')
df1.emp_length = df1.emp_length.str.replace('years', '')
df1.emp_length = df1.emp_length.str.replace('year', '')

In [130]:
df1.emp_length = pd.to_numeric(df1.emp_length)

In [131]:
df1.emp_length.unique()

array([10,  3,  4,  6,  7,  8,  2,  5,  9,  1], dtype=int64)

In [132]:
dfo = df1.select_dtypes(include = "object")

In [133]:
dfo.head()

,grade,sub_grade,home_ownership,verification_status,issue_d,pymnt_plan,purpose,title,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
0,C,C4,MORTGAGE,Not Verified,Dec-2015,n,debt_consolidation,Debt consolidation,PA,Aug-2003,w,Jan-2019,Mar-2019,Individual,N,Cash,N
1,C,C1,MORTGAGE,Not Verified,Dec-2015,n,small_business,Business,SD,Dec-1999,w,Jun-2016,Mar-2019,Individual,N,Cash,N
3,F,F1,MORTGAGE,Source Verified,Dec-2015,n,major_purchase,Major purchase,PA,Jun-1998,w,Jul-2016,Mar-2018,Individual,N,Cash,N
4,C,C3,RENT,Source Verified,Dec-2015,n,debt_consolidation,Debt consolidation,GA,Oct-1987,w,May-2017,May-2017,Individual,N,Cash,N
5,B,B2,MORTGAGE,Not Verified,Dec-2015,n,debt_consolidation,Debt consolidation,MN,Jun-1990,f,Nov-2016,Mar-2019,Individual,N,Cash,N


In [134]:
df1.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
   

In [135]:
dfo.home_ownership.unique()

array(['MORTGAGE', 'RENT', 'OWN', 'ANY', 'NONE', 'OTHER'], dtype=object)

In [136]:
df1.collections_12_mths_ex_med.unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6., 14., 16., 20., 10.])

### Let's decide to drop some of the categorical features that may not be useful for our purposes:

In [173]:
drop_list = ["pymnt_plan","sub_grade", "issue_d", "title", "addr_state", "earliest_cr_line", "last_pymnt_d", "initial_list_status", \
            "last_credit_pull_d", "disbursement_method", "debt_settlement_flag", "verification_status", "hardship_flag", \
             "loan_amnt", "funded_amnt_inv", "fico_range_high", "fico_range_low", "pub_rec", "inq_last_6mths", "recoveries", \
            "collection_recovery_fee", "last_fico_range_high", "last_fico_range_low", "collections_12_mths_ex_med", "policy_code", \
            "mo_sin_old_il_acct", "total_rec_prncp", "last_pymnt_amnt", "total_pymnt", "out_prncp_inv", "out_prncp", \
             "total_rec_prncp", "int_rate", "chargeoff_within_12_mths", "acc_now_delinq", "earliest_cr_line", "total_pymnt_inv", \
            "total_rec_int", "total_rec_late_fee"]             

In [174]:
df2 = df1.drop(drop_list, axis = 1)

In [175]:
df2.head()

,funded_amnt,term,installment,grade,emp_length,home_ownership,annual_inc,loan_status,purpose,dti,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,3600.0,36,123.03,C,10,MORTGAGE,55000.0,0,debt_consolidation,5.91,...,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0
1,24700.0,36,820.28,C,10,MORTGAGE,65000.0,0,small_business,16.06,...,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0
3,10400.0,60,289.91,F,3,MORTGAGE,104433.0,0,major_purchase,25.37,...,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0
4,11950.0,36,405.18,C,4,RENT,34000.0,0,debt_consolidation,10.20,...,0.0,0.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0
5,20000.0,36,637.58,B,10,MORTGAGE,180000.0,0,debt_consolidation,14.67,...,0.0,2.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0


In [176]:
df2.select_dtypes(include = "object")

,grade,home_ownership,purpose,application_type
0,C,MORTGAGE,debt_consolidation,Individual
1,C,MORTGAGE,small_business,Individual
3,F,MORTGAGE,major_purchase,Individual
4,C,RENT,debt_consolidation,Individual
5,B,MORTGAGE,debt_consolidation,Individual
...,...,...,...,...
1852315,C,MORTGAGE,debt_consolidation,Individual
1852318,C,MORTGAGE,debt_consolidation,Individual
1852319,C,MORTGAGE,home_improvement,Individual
1852320,C,RENT,medical,Individual


In [177]:
df2.application_type.value_counts()

Individual    1070305
Joint App       20336
Name: application_type, dtype: int64

## Let's drop "Loan_status" colum from dataframe and save it as a y variable

In [178]:
y=df2["loan_status"]

In [179]:
df2.drop(["loan_status"],axis =1, inplace = True)

In [180]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df2.copy()
numeric_cols = list(df_scaled.dtypes[df_scaled.dtypes != 'object'].index)
df_scaled.loc[:,numeric_cols] = scaler.fit_transform(df_scaled.loc[:,numeric_cols])

In [181]:
df_scaled.head()

,funded_amnt,term,installment,grade,emp_length,home_ownership,annual_inc,purpose,dti,delinq_2yrs,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,-1.279802,-0.579069,-1.240009,C,1.098272,MORTGAGE,-0.328292,debt_consolidation,-1.324187,-0.367941,...,-0.171637,0.447711,-2.043660,-1.252435,-0.345709,-0.132342,-0.010868,-0.913761,-0.917892,-0.706974
1,1.128241,-0.579069,1.415401,C,1.098272,MORTGAGE,-0.189794,small_business,-0.260251,0.756352,...,-0.171637,-0.097711,0.359521,-1.038034,-0.345709,-0.132342,0.755962,-0.262861,2.667526,-0.457813
3,-0.503750,1.726909,-0.604462,F,-0.865783,MORTGAGE,0.356344,major_purchase,0.715635,0.756352,...,-0.171637,0.993133,0.265738,0.418221,-0.345709,-0.132342,1.464059,0.891953,-0.083315,0.987743
4,-0.326856,-0.579069,-0.165468,C,-0.585204,RENT,-0.619137,debt_consolidation,-0.874504,-0.367941,...,-0.171637,-1.188556,0.664315,1.531992,-0.345709,-0.132342,-0.919727,-0.810122,-0.591521,-0.928810
5,0.591852,-0.579069,0.719605,B,1.098272,MORTGAGE,1.402929,debt_consolidation,-0.405953,-0.367941,...,-0.171637,-0.097711,0.230570,1.531992,-0.345709,-0.132342,1.178019,1.322631,0.438878,0.038662


In [182]:
df_scaled_dum = pd.get_dummies(df_scaled)

In [183]:
df_scaled_dum.head()

,funded_amnt,term,installment,emp_length,annual_inc,dti,delinq_2yrs,open_acc,revol_bal,revol_util,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Individual,application_type_Joint App
0,-1.279802,-0.579069,-1.240009,1.098272,-0.328292,-1.324187,-0.367941,-0.896622,-0.630601,-0.912299,...,0,0,0,0,0,0,0,0,1,0
1,1.128241,-0.579069,1.415401,1.098272,-0.189794,-0.260251,0.756352,1.830533,0.232760,-1.345031,...,0,0,0,0,0,1,0,0,1,0
3,-0.503750,1.726909,-0.604462,-0.865783,0.356344,0.715635,0.756352,0.012430,0.253946,0.521900,...,1,0,0,0,0,0,0,0,1,0
4,-0.326856,-0.579069,-0.165468,-0.585204,-0.619137,-0.874504,-0.367941,-1.260242,-0.351030,0.682629,...,0,0,0,0,0,0,0,0,1,0
5,0.591852,-0.579069,0.719605,1.098272,1.402929,-0.405953,-0.367941,0.012430,3.272592,1.346153,...,0,0,0,0,0,0,0,0,1,0


In [184]:
X = df_scaled_dum.copy()

### It seems like the data is normalized and dummy variable is created for categorical features. Now, Lets, split the data into training and testing sets and get ready to apply machine learning algorithms.

In [185]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [186]:
print("Shape of X_train:", np.shape(X_train))
print("Shape of y_train:", np.shape(y_train))
print()
print("Shape of X_test:", np.shape(X_test))
print("Shape of y_test:", np.shape(y_test))

Shape of X_train: (817980, 75)
Shape of y_train: (817980,)

Shape of X_test: (272661, 75)
Shape of y_test: (272661,)


In [187]:
X["loan_status"] = y

In [188]:
X.head()

,funded_amnt,term,installment,emp_length,annual_inc,dti,delinq_2yrs,open_acc,revol_bal,revol_util,...,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Individual,application_type_Joint App,loan_status
0,-1.279802,-0.579069,-1.240009,1.098272,-0.328292,-1.324187,-0.367941,-0.896622,-0.630601,-0.912299,...,0,0,0,0,0,0,0,1,0,0
1,1.128241,-0.579069,1.415401,1.098272,-0.189794,-0.260251,0.756352,1.830533,0.232760,-1.345031,...,0,0,0,0,1,0,0,1,0,0
3,-0.503750,1.726909,-0.604462,-0.865783,0.356344,0.715635,0.756352,0.012430,0.253946,0.521900,...,0,0,0,0,0,0,0,1,0,0
4,-0.326856,-0.579069,-0.165468,-0.585204,-0.619137,-0.874504,-0.367941,-1.260242,-0.351030,0.682629,...,0,0,0,0,0,0,0,1,0,0
5,0.591852,-0.579069,0.719605,1.098272,1.402929,-0.405953,-0.367941,0.012430,3.272592,1.346153,...,0,0,0,0,0,0,0,1,0,0


In [189]:
## save data for next step
from sb_utils import save_file
data_dir = "./"
save_file(X, "LendingClub_scaled.csv", data_dir)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "./LendingClub_scaled.csv"


## Now, go to modeling part